In [ ]:
import datajoint as dj
img= dj.schema('arseny_learning_imaging', locals())
imgframstarttrail1= dj.VirtualModule('IMG', 'arseny_learning_imaging')
imgframstarttrail= imgframstarttrail1.IMG.FramStartTrail

tracking= dj.schema('arseny_learning_tracking', locals())
tracking= dj.VirtualModule('TRACKING', 'arseny_learning_tracking')
grooming= tracking.VideoGroomingTrail

exp2= dj.schema('arseny_s1alm_experiment2', locals())
exp2vm= dj.VirtualModule ('EXP2', 'arseny_s1alm_experiment2')
behaviortrail= exp2vm.BehaviorTrial
behaviortrailevent= exp2vm.BehaviorTrailEvent
actionevent= exp2vm.ActionEvent

lick2dtalp= dj.schema('talch012_lick2dtalp', locals())
lick2dtal= dj.VirtualModule('Lick2DtalP', 'talch012_lick2dtalp')

from fn_parse_into_trials_and_get_lickrate_tal_1 import fn_parse_into_trials_and_get_lickrate_tal_1
from lick2dtalp.ROILick2DPSTHSpikes import rel_data, rel_temp, flag_eletric_video, fr_interval_limit, fr_interval, time_resample_bin 
# real_data comes from img.roispike that has the key 'spikes_trace'= deconvolved trace for each frame

# discard the first and last trails
# session epoch trail comes from the table in the same name that describe behave only, behav photo, spint only and spont photo
# whould it be esier to import the psth_mean_stem function?

In [ ]:
## table keys called in this function

# IMG.ROI
#   roi_number, roi_number_uid (=unique across sessions, special cell we are following throughout exp), roi_centroid_x,roi_centroid_y (=the intersect of the triangle medians x comp and y comp), roi_radius, roi_x_pix, roi_y_pix, roi_pixel_weight...
# IMG.ROIBad
#   bad ROI that were considered cell by suite2p
# FOVEpoch
#   imaging_fram_rate, "_"_"_plane, "_"_"_volume, zoom,imaging_fov_deg, imaging_fov_um
# FOV
#   fov_name, fov_x_size (=pixels), fov_y_size (=pixels), imaging_fram_rate (redundant


In [ ]:
def fn_computer_Lick2DPSTH(key, self, rel_data, fr_interval, fr_interval_limit, flag_electric_video, time_resample_bin, self2, self3, self4):
    import numpy as np
    import scipy as scipy
    import scipy.stats
    from scipy import signal
    from scipy import stats as st
    from scipy.stats import ranksums, pearsonr
    from datajoint import fetch1, fetch, insert
    
    # frames for PSTH
    smooth_window_sec = 0.2

    # fetch data
    rel_ROI = (img.ROI - img.ROIBad) & key # roi bad are bad ROI that were considered cell by suite2p
    key_ROI1 = fetch(rel_ROI, order_by='roi_number')  # lick2dtalp.ROILick2DPSTHSpikes
    key_ROI2 = fetch(rel_ROI, order_by='roi_number')  # lick2dtalp.ROILick2DPSTHStatsSpikes
    key_ROI3 = fetch(rel_ROI, order_by='roi_number')  # lick2dtalp.ROILick2DPSTHBlockSpikes
    key_ROI4 = fetch(rel_ROI, order_by='roi_number')  # lick2dtalp.ROILick2DPSTHBlockStatsSpikes
    rel_data = rel_data & rel_ROI & key
    try:
        frame_rate = fetch1(img.FOVEpoch & key, 'imaging_frame_rate')
    except:
        frame_rate = fetch1(img.FOV & key, 'imaging_frame_rate')
    smooth_window_frames = int(np.ceil(smooth_window_sec * frame_rate))  # frames for PSTH, Rounds toward positive infinity 

    # fetch trial data
    R = fetch((exp2.TrialRewardSize & key) - tracking.VideoGroomingTrial, '*', order_by='trial') # It fetches the info from all the keys in the immediate table and all the primary keys from the parental tables -> from session trail it takes 'trail', from sessioms it takes 'session' and from subject it taks 'subject id' and excludes the times grooming obsreved
    block = fetch((exp2.TrialLickBlock & key) - tracking.VideoGroomingTrial, '*', order_by='trial')
    S = fetch(rel_data, '*') #rel_data comes from the table definition, it refers to ROISpikes (flourescent trace) that calculates the spikes_trace which is deconvolved trace for each frame 
    if 'spikes_trace' in S:  # to be able to run the code both on dff and on deconvulted "spikes" data
        S['dff_trace'] = S['spikes_trace'] #dff_trace is a defined key
        del S['spikes_trace']
    
    # parsing trials
    start_file, end_file = fn_parse_into_trials_and_get_lickrate_tal_1(key, frame_rate, fr_interval, flag_electric_video)
    start_file[-1], end_file[-1] = np.nan, np.nan  # we don't use the very last trial
    num_trials = len(start_file)
    idx_response = ~np.isnan(start_file)
    
    try:
        # idx reward
        #idx_regular = np.where(np.array([R['reward_size_type'] == 'regular' for trial in R]) & idx_response)[0]
        idx_regular = np.where(np.array([R[i]['reward_size_type'] == 'regular' for i in range(num_trials)]) & idx_response)[0] # fond the index of regular, small and large 
        idx_regular_temp = np.array([R[i]['reward_size_type'] == 'regular' for i in range(num_trials)]) & idx_response # regular ind by 01 
        idx_small = np.where(np.array([R[i]['reward_size_type'] == 'omission' for i in range(num_trials)]) & idx_response)[0]
        idx_large = np.where(np.array([R[i]['reward_size_type'] == 'large' for i in range(num_trials)]) & idx_response)[0]
        idx_odd_small = idx_small[::2][:len(idx_small)] # chooses the odd out of the small  from the regular 
        idx_even_small = idx_small[1::2][:len(idx_small)]
        idx_odd_large = idx_large[::2][:len(idx_large)]
        idx_even_large = idx_large[1::2][:len(idx_large)]
    except:
        idx_regular = np.where(np.arange(1, num_trials + 1) & idx_response)[0]
        idx_regular_temp = np.arange(1, num_trials + 1)[idx_response]

    idx_odd_regular = idx_regular[::2][:len(idx_regular)]
    idx_even_regular = idx_regular[1::2][:len(idx_regular)]
    
    try
        # devied the block to 4 bins according to the current_trial_number_in_block- the first trial are defined as first all the first trials, and the three others inbetween the bins values defined
        # find the most frequent num_trials_in_block. in matlab we use mode(), in python we can use statistics.mode() but here we use bincount to count each value and returns the highest count value. note that in both (python&matlab) cases it returns the first mode he founds (meaning if you have two most freq values, it returns the first that appeared) 
        # we devied the regular but no small/large as regular are 80% of the data. 
        num_trials_in_block = np.bincount(block.num_trials_in_block).argmax()
        begin_mid_end_bins = np.linspace(2, num_trials_in_block, 4)# returns 4 values the first is 2, the last is num_trials_in_block and the two middel ones are num_trials_in_block-(num_trials_in_block-2)/3 and ans-(num_trials_in_block-2)/3

        idx_first = np.where((np.array(Block.current_trial_num_in_block) == 1) & idx_response & idx_regular_temp)[0] #indexes of the current trail in block==1&not nan& ind of the 01 map of regular reward
        idx_begin = np.where((np.array(Block.current_trial_num_in_block) >= begin_mid_end_bins[0]) & (np.array(Block.current_trial_num_in_block) <= np.floor(begin_mid_end_bins[1])) & idx_response & idx_regular_temp)[0]
        idx_mid = np.where((np.array(Block.current_trial_num_in_block) > begin_mid_end_bins[1]) & (np.array(Block.current_trial_num_in_block) <= np.round(begin_mid_end_bins[2])) & idx_response & idx_regular_temp)[0]
        idx_end = np.where((np.array(Block.current_trial_num_in_block) > begin_mid_end_bins[2]) & (np.array(Block.current_trial_num_in_block) <= np.ceil(begin_mid_end_bins[3])) & idx_response & idx_regular_temp)[0]

        idx_odd_first = idx_first[::2][np.in1d(idx_regular[::2], idx_first)]
        idx_even_first = idx_first[1::2][np.in1d(idx_regular[1::2], idx_first)]

        idx_odd_begin = idx_begin[::2][np.in1d(idx_regular[::2], idx_begin)]
        idx_even_begin = idx_begin[1::2][np.in1d(idx_regular[1::2], idx_begin)]

        idx_odd_mid = idx_mid[::2][np.in1d(idx_regular[::2], idx_mid)]
        idx_even_mid = idx_mid[1::2][np.in1d(idx_regular[1::2], idx_mid)]

        idx_odd_end = idx_end[::2][np.in1d(idx_regular[::2], idx_end)]
        idx_even_end = idx_end[1::2][np.in1d(idx_regular[1::2], idx_end)]
    except
        # add an exception
    
    # we take the session epoch type and number of the current key (one iteration) and apply it to the coresponding possition in each key_ROI(i). we are going to use key_ROI for####
    for i_roi in range(1, S.shape[0] + 1):
    key_ROI1 = dict(session_epoch_type=key['session_epoch_type'], session_epoch_number=key['session_epoch_number'])
    key_ROI2 = dict(session_epoch_type=key['session_epoch_type'], session_epoch_number=key['session_epoch_number'])
    key_ROI3 = dict(session_epoch_type=key['session_epoch_type'], session_epoch_number=key['session_epoch_number'])
    key_ROI4 = dict(session_epoch_type=key['session_epoch_type'], session_epoch_number=key['session_epoch_number'])

    # Use key_ROI1, key_ROI2, key_ROI3, key_ROI4 as needed
    # Perform further operations or insert into DataJoint tables

    #PSTH 
    psth_all = [None] * len(i_tr)
    spikes = S[i_roi]['dff_trace'] #spikes is the dff_trace of each row in S
    # time_resample_bin is originated from the tables definition, and it is empty
    if time_resample_bin is None:
        for i_tr in range(len(start_file)):
            if idx_response[i_tr] == 0:  # its an ignore trial
                psth_all[i_tr] = None
                continue
        
            s = spikes[start_file[i_tr]:end_file[i_tr]]
            s = np.convolve(s, np.ones(smooth_window_frames) / smooth_window_frames, mode='valid')
            # s_interval=s(time>fr_interval(1) & time<=fr_interval(2));
            # fr_all(i_roi,i_tr)= max(s_interval); %taking the max
            time = np.arange(1, len(s) + 1) / frame_rate + fr_interval[0]
            psth_all[i_tr] = s
    else:
        time_new_bins = np.arange(fr_interval[0], fr_interval[-1], time_resample_bin)
        time_new = time_new_bins[:-1] + np.mean(np.diff(time_new_bins)) / 2
        for i_tr in range(len(start_file)):
            if idx_response[i_tr] == 0:
                psth_all[i_tr] = None
                continue
            s = spikes[start_file[i_tr]:end_file[i_tr]]
            s = np.convolve(s, np.ones(smooth_window_frames) / smooth_window_frames, mode='valid')
            time = np.arange(1, len(s) + 1) / frame_rate + fr_interval[0]
            s_resampled = []
            for i_t in range(len(time_new_bins) - 1):
                idx_t = (time > time_new_bins[i_t]) & (time <= time_new_bins[i_t + 1])
                s_resampled.append(np.mean(s[idx_t]))
            psth_all[i_tr] = s_resampled
        time = time_new


    psth_regular = np.mean(np.concatenate(psth_all[idx_regular]), axis=0)
    psth_regular_stem = np.std(np.concatenate(psth_all[idx_regular]), axis=0) / np.sqrt(len(idx_regular))
    psth_regular_odd = np.nanmean(np.concatenate(psth_all[idx_odd_regular]), axis=0)
    psth_regular_even = np.nanmean(np.concatenate(psth_all[idx_even_regular]), axis=0)

    key_ROI1[i_roi].insert1({
        'psth_regular': psth_regular,
        'psth_regular_stem': psth_regular_stem,
        'psth_regular_odd': psth_regular_odd,
        'psth_regular_even': psth_regular_even,
        'psth_time': time
    })

    # Calculating Pearson correlation for odd vs even to see if the data matches itself
    r = np.corrcoef(psth_regular_odd, psth_regular_even)[0, 1]
    key_ROI2[i_roi].insert1({'psth_regular_odd_vs_even_corr': r})

    # Identifying the peak in the trace and when it occurred for the whole data
    idx_peak = np.argmax(psth_regular)
    key_ROI2[i_roi].insert1({'peaktime_psth_regular': time[idx_peak]})

    idx_peak = np.argmax(psth_regular_odd)
    key_ROI2[i_roi].insert1({'peaktime_psth_regular_odd': time[idx_peak]})

    idx_peak = np.argmax(psth_regular_even)
    key_ROI2[i_roi].insert1({'peaktime_psth_regular_even': time[idx_peak]})

    try:
        # Reward
        # Taking the mean PSTH across trials
        psth_small = np.nanmean(np.concatenate(psth_all[idx_small]), axis=0)
        psth_small_stem = np.std(np.concatenate(psth_all[idx_small]), axis=0) / np.sqrt(len(idx_small))
        psth_small_odd = np.nanmean(np.concatenate(psth_all[idx_odd_small]), axis=0)
        psth_small_even = np.nanmean(np.concatenate(psth_all[idx_even_small]), axis=0)

        psth_large = np.nanmean(np.concatenate(psth_all[idx_large]), axis=0)
        psth_large_stem = np.std(np.concatenate(psth_all[idx_large]), axis=0) / np.sqrt(len(idx_large))
        psth_large_odd = np.nanmean(np.concatenate(psth_all[idx_odd_large]), axis=0)
        psth_large_even = np.nanmean(np.concatenate(psth_all[idx_even_large]), axis=0)

        key_ROI1[i_roi].psth_small = psth_small
        key_ROI1[i_roi].psth_small_stem = psth_small_stem
        key_ROI1[i_roi].psth_small_odd = psth_small_odd
        key_ROI1[i_roi].psth_small_even = psth_small_even

        key_ROI1[i_roi].psth_large = psth_large
        key_ROI1[i_roi].psth_large_stem = psth_large_stem
        key_ROI1[i_roi].psth_large_odd = psth_large_odd
        key_ROI1[i_roi].psth_large_even = psth_large_even


        r = np.corrcoef(psth_small_odd, psth_small_even)[0, 1]
        key_ROI2[i_roi].psth_small_odd_vs_even_corr = r

        r = np.corrcoef(psth_large_odd, psth_large_even)[0, 1]
        key_ROI2[i_roi].psth_large_odd_vs_even_corr = r

        r = np.corrcoef(psth_regular, psth_small)[0, 1]
        key_ROI2[i_roi].psth_regular_vs_small_corr = r

        r = np.corrcoef(psth_regular, psth_large)[0, 1]
        key_ROI2[i_roi].psth_regular_vs_large_corr = r

        r = np.corrcoef(psth_small, psth_large)[0, 1]
        key_ROI2[i_roi].psth_small_vs_large_corr = r


        idx_peak_small = np.argmax(psth_small)
        key_ROI2[i_roi].peaktime_psth_small = time[idx_peak_small]
        idx_peak_regular = np.argmax(psth_regular)
        key_ROI2[i_roi].peaktime_psth_regular = time[idx_peak_regular]
        idx_peak_large = np.argmax(psth_large)
        key_ROI2[i_roi].peaktime_psth_large = time[idx_peak_large]

        # single trials, averaged across all time duration in a specific time interval (e.g. after the licport onset (t>=0))
        idx_onset = (time >= fr_interval_limit[0]) & (time < fr_interval_limit[1])
        temp = np.concatenate(psth_all[idx_regular])
        psth_regular_trials = np.nanmean(temp[:, idx_onset], axis=1)
        temp = np.concatenate(psth_all[idx_small])
        psth_small_trials = np.nanmean(temp[:, idx_onset], axis=1)
        temp = np.concatenate(psth_all[idx_large])
        psth_large_trials = np.nanmean(temp[:, idx_onset], axis=1)

        p_regular_small, _ = ranksum(psth_regular_trials, psth_small_trials)
        key_ROI2[i_roi].reward_mean_pval_regular_small = p_regular_small
        p_regular_large, _ = ranksum(psth_regular_trials, psth_large_trials)
        key_ROI2[i_roi].reward_mean_pval_regular_large = p_regular_large
        p_small_large, _ = ranksum(psth_small_trials, psth_large_trials)
        key_ROI2[i_roi].reward_mean_pval_small_large = p_small_large

        key_ROI2[i_roi].reward_mean_regular = np.nanmean(psth_regular_trials)
        key_ROI2[i_roi].reward_mean_small = np.nanmean(psth_small_trials)
        key_ROI2[i_roi].reward_mean_large = np.nanmean(psth_large_trials)

        # at peak response time
        temp = np.concatenate(psth_all[idx_regular])
        psth_regular_trials_peak = temp[:, idx_peak_regular]
        temp = np.concatenate(psth_all[idx_small])
        psth_small_trials_peak = temp[:, idx_peak_small]
        temp = np.concatenate(psth_all[idx_large])
        psth_large_trials_peak = temp[:, idx_peak_large]

        key_ROI2[i_roi].reward_peak_regular = np.nanmean(psth_regular_trials_peak)
        key_ROI2[i_roi].reward_peak_small = np.nanmean(psth_small_trials_peak)
        key_ROI2[i_roi].reward_peak_large = np.nanmean(psth_large_trials_peak)

        p_peak_regular_small, _ = ranksum(psth_regular_trials_peak, psth_small_trials_peak)
        key_ROI2[i_roi].reward_peak_pval_regular_small = p_peak_regular_small
        p_peak_regular_large, _ = ranksum(psth_regular_trials_peak, psth_large_trials_peak)
        key_ROI2[i_roi].reward_peak_pval_regular_large = p_peak_regular_large
        p_peak_small_large, _ = ranksum(psth_small_trials_peak, psth_large_trials_peak)
        key_ROI2[i_roi].reward_peak_pval_small_large = p_peak_small_large

        # k2 = key_ROI3[i_roi]
        # insert(self3, k2)
    except:
        pass

    try:
        # BLOCK
        # Taking the mean PSTH across trials
        psth_first = np.nanmean(np.concatenate(psth_all[idx_first]), axis=0)
        psth_first_stem = np.std(np.concatenate(psth_all[idx_first]), axis=0) / np.sqrt(len(idx_first))
        psth_first_odd = np.nanmean(np.concatenate(psth_all[idx_odd_first]), axis=0)
        psth_first_even = np.nanmean(np.concatenate(psth_all[idx_even_first]), axis=0)

        psth_begin = np.nanmean(np.concatenate(psth_all[idx_begin]), axis=0)
        psth_begin_stem = np.std(np.concatenate(psth_all[idx_begin]), axis=0) / np.sqrt(len(idx_begin))
        psth_begin_odd = np.nanmean(np.concatenate(psth_all[idx_odd_begin]), axis=0)
        psth_begin_even = np.nanmean(np.concatenate(psth_all[idx_even_begin]), axis=0)

        psth_mid = np.nanmean(np.concatenate(psth_all[idx_mid]), axis=0)
        psth_mid_stem = np.std(np.concatenate(psth_all[idx_mid]), axis=0) / np.sqrt(len(idx_mid))
        psth_mid_odd = np.nanmean(np.concatenate(psth_all[idx_odd_mid]), axis=0)
        psth_mid_even = np.nanmean(np.concatenate(psth_all[idx_even_mid]), axis=0)

        psth_end = np.nanmean(np.concatenate(psth_all[idx_end]), axis=0)
        psth_end_stem = np.std(np.concatenate(psth_all[idx_end]), axis=0) / np.sqrt(len(idx_end))
        psth_end_odd = np.nanmean(np.concatenate(psth_all[idx_odd_end]), axis=0)
        psth_end_even = np.nanmean(np.concatenate(psth_all[idx_even_end]), axis=0)

        key_ROI1[i_roi].psth_first = psth_first
        key_ROI1[i_roi].psth_first_stem = psth_first_stem
        key_ROI1[i_roi].psth_first_odd = psth_first_odd
        key_ROI1[i_roi].psth_first_even = psth_first_even

        key_ROI1[i_roi].psth_begin = psth_begin
        key_ROI1[i_roi].psth_begin_stem = psth_begin_stem
        key_ROI1[i_roi].psth_begin_odd = psth_begin_odd
        key_ROI1[i_roi].psth_begin_even = psth_begin_even

        key_ROI1[i_roi].psth_mid = psth_mid
        key_ROI1[i_roi].psth_mid_stem = psth_mid_stem
        key_ROI1[i_roi].psth_mid_odd = psth_mid_odd
        key_ROI1[i_roi].psth_mid_even = psth_mid_even

        key_ROI1[i_roi].psth_end = psth_end
        key_ROI1[i_roi].psth_end_stem = psth_end_stem
        key_ROI1[i_roi].psth_end_odd = psth_end_odd
        key_ROI1[i_roi].psth_end_even = psth_end_even

        key_ROI3[i_roi].psth_first = psth_first
        key_ROI3[i_roi].psth_first_stem = psth_first_stem
        key_ROI3[i_roi].psth_first_odd = psth_first_odd
        key_ROI3[i_roi].psth_first_even = psth_first_even

        key_ROI3[i_roi].psth_begin = psth_begin
        key_ROI3[i_roi].psth_begin_stem = psth_begin_stem
        key_ROI3[i_roi].psth_begin_odd = psth_begin_odd
        key_ROI3[i_roi].psth_begin_even = psth_begin_even

        key_ROI3[i_roi].psth_mid = psth_mid
        key_ROI3[i_roi].psth_mid_stem = psth_mid_stem
        key_ROI3[i_roi].psth_mid_odd = psth_mid_odd
        key_ROI3[i_roi].psth_mid_even = psth_mid_even

        key_ROI3[i_roi].psth_end = psth_end
        key_ROI3[i_roi].psth_end_stem = psth_end_stem
        key_ROI3[i_roi].psth_end_odd = psth_end_odd
        key_ROI3[i_roi].psth_end_even = psth_end_even

        # Stability
        r = np.corrcoef([psth_first_odd.flatten(), psth_first_even.flatten()], rowvar=False)
        key_ROI4[i_roi].psth_first_odd_vs_even_corr = r[0, 1]

        r = np.corrcoef([psth_begin_odd.flatten(), psth_begin_even.flatten()], rowvar=False)
        key_ROI4[i_roi].psth_begin_odd_vs_even_corr = r[0, 1]

        r = np.corrcoef([psth_mid_odd.flatten(), psth_mid_even.flatten()], rowvar=False)
        key_ROI4[i_roi].psth_mid_odd_vs_even_corr = r[0, 1]

        r = np.corrcoef([psth_end_odd.flatten(), psth_end_even.flatten()], rowvar=False)
        key_ROI4[i_roi].psth_end_odd_vs_even_corr = r[0, 1]

        # Between conditions
        r = np.corrcoef([psth_first.flatten(), psth_begin.flatten()], rowvar=False)
        key_ROI4[i_roi].psth_first_vs_begin_corr = r[0, 1]

        r = np.corrcoef([psth_first.flatten(), psth_mid.flatten()], rowvar=False)
        key_ROI4[i_roi].psth_first_vs_mid_corr = r[0, 1]

        r = np.corrcoef([psth_first.flatten(), psth_end.flatten()], rowvar=False)
        key_ROI4[i_roi].psth_first_vs_end_corr = r[0, 1]

        r = np.corrcoef([psth_begin.flatten(), psth_end.flatten()], rowvar=False)
        key_ROI4[i_roi].psth_begin_vs_end_corr = r[0, 1]

        r = np.corrcoef([psth_begin.flatten(), psth_mid.flatten()], rowvar=False)
        key_ROI4[i_roi].psth_begin_vs_mid_corr = r[0, 1]

        r = np.corrcoef([psth_mid.flatten(), psth_end.flatten()], rowvar=False)
        key_ROI4[i_roi].psth_mid_vs_end_corr = r[0, 1]

        _, idx_peak_first = np.max(psth_first)
        key_ROI4[i_roi].peaktime_psth_first = time[idx_peak_first]
        _, idx_peak_begin = np.max(psth_begin)
        key_ROI4[i_roi].peaktime_psth_begin = time[idx_peak_begin]
        _, idx_peak_mid = np.max(psth_mid)
        key_ROI4[i_roi].peaktime_psth_mid = time[idx_peak_mid]
        _, idx_peak_end = np.max(psth_end)
        key_ROI4[i_roi].peaktime_psth_end = time[idx_peak_end]

        # Single trials, averaged across all time duration in a specific time interval (e.g., after the licport onset (t>=0))
        idx_onset = (time >= fr_interval_limit[0]) & (time < fr_interval_limit[1])
        temp = np.concatenate(psth_all[idx_first], axis=0)
        psth_first_trials = np.nanmean(temp[:, idx_onset], axis=1)
        temp = np.concatenate(psth_all[idx_begin], axis=0)
        psth_begin_trials = np.nanmean(temp[:, idx_onset], axis=1)
        temp = np.concatenate(psth_all[idx_mid], axis=0)
        psth_mid_trials = np.nanmean(temp[:, idx_onset], axis=1)
        temp = np.concatenate(psth_all[idx_end], axis=0)
        psth_end_trials = np.nanmean(temp[:, idx_onset], axis=1)

        key_ROI4[i_roi].block_mean_first = np.nanmean(psth_first_trials)
        key_ROI4[i_roi].block_mean_begin = np.nanmean(psth_begin_trials)
        key_ROI4[i_roi].block_mean_mid = np.nanmean(psth_mid_trials)
        key_ROI4[i_roi].block_mean_end = np.nanmean(psth_end_trials)

        p, _ = scipy.stats.ranksums(psth_first_trials, psth_begin_trials)
        key_ROI4[i_roi].block_mean_pval_first_begin = p
        p, _ = scipy.stats.ranksums(psth_first_trials, psth_end_trials)
        key_ROI4[i_roi].block_mean_pval_first_end = p
        p, _ = scipy.stats.ranksums(psth_begin_trials, psth_end_trials)
        key_ROI4[i_roi].block_mean_pval_begin_end = p

        # At peak response time
        temp = np.concatenate(psth_all[idx_first], axis=0)
        psth_first_trials_peak = temp[:, idx_peak_first]
        temp = np.concatenate(psth_all[idx_begin], axis=0)
        psth_begin_trials_peak = temp[:, idx_peak_begin]
        temp = np.concatenate(psth_all[idx_mid], axis=0)
        psth_mid_trials_peak = temp[:, idx_peak_mid]
        temp = np.concatenate(psth_all[idx_end], axis=0)
        psth_end_trials_peak = temp[:, idx_peak_end]

        key_ROI4[i_roi].block_peak_first = np.nanmean(psth_first_trials_peak)
        key_ROI4[i_roi].block_peak_begin = np.nanmean(psth_begin_trials_peak)
        key_ROI4[i_roi].block_peak_mid = np.nanmean(psth_mid_trials_peak)
        key_ROI4[i_roi].block_peak_end = np.nanmean(psth_end_trials_peak)

        p, _ = scipy.stats.ranksums(psth_first_trials_peak, psth_begin_trials_peak)
        key_ROI4[i_roi].block_peak_pval_first_begin = p
        p, _ = scipy.stats.ranksums(psth_first_trials_peak, psth_end_trials_peak)
        key_ROI4[i_roi].block_peak_pval_first_end = p
        p, _ = scipy.stats.ranksums(psth_begin_trials_peak, psth_end_trials_peak)
        key_ROI4[i_roi].block_peak_pval_begin_end = p

        # k2=key_ROI4[i_roi]
        # insert(self4, k2)
       
    except:
        pass

  
    # Inserting bulk data into tables
    self.insert(key_ROI1)  
    self2.insert(key_ROI2)  

    try:
        # Insert into key_ROI3 table
        self3.insert(key_ROI3) 
    except:
        pass

    try:
    # Insert into key_ROI4 table
        self4.insert(key_ROI4)  
    except:
        pass

